<a href="https://colab.research.google.com/github/Omodaka9375/AI-DocProduct/blob/master/DocProductPresentationV6_0_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![DoctorBert](https://snag.gy/oF84Gk.jpg)


![DoctorBert](https://i.ytimg.com/vi/nPemP-Q0Xn8/hqdefault.jpg)

This is a Colab Demo of our DocProduct Tensorflow 2.0 Hackathon Project

Project details can be seen on our Github repo
https://github.com/Santosh-Gupta/DocProduct

and our Devpost page
https://devpost.com/software/nlp-doctor

In order to fit the computational limits of Google Colab, we had to reduce the amount of searchable Q&A datapoints from ~700,000 to ~200,000.
We also converted our search embeddings from float32 to float16, although this does not seem to dimish the quality of search (*but that's a whole other product to explore*)

Instructions:

-Run the first cell to install TF2.0 and FAISS

-Run the second cell to download the models and data

-Run the third cell to load the weights and data

-Input your question and search parameters into the fourth cell to play around with our model results




In [0]:
!pip install q keras==2.3.1

In [0]:
%tensorflow_version 2.x

In [1]:
#@title Install Faiss, TF 2.0, and our Github. Double Click to see code

#To use CPU FAISS use
!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
#To use GPU FAISS use
# !wget  https://anaconda.org/pytorch/faiss-gpu/1.2.1/download/linux-64/faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

!pip install tensorflow==2.0.0-alpha0
import tensorflow as tf
!pip install https://github.com/re-search/DocProduct/archive/v0.2.0_dev.zip
!pip install gpt2-estimator
!pip install pyarrow


--2020-04-22 22:59:05--  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.92.24, 104.17.93.24, 2606:4700::6811:5d18, ...
Connecting to anaconda.org (anaconda.org)|104.17.92.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9c5c376961204909d87/5aa7f0a65571b411e5c259be?response-content-disposition=attachment%3B%20filename%3D%22faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20200422T225905Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=IQoJb3JpZ2luX2VjECQaCXVzLWVhc3QtMSJHMEUCIQDKBNgCgnYUBLGGHovmc3eJQdijJ0Y9cgjphO30UdvY2wIgCtzmVf9LRm8CQ5y2WlDreyvAhNYx4nm1cog4DUXVvk4qtAMITRAAGgw0NTU4NjQwOTgzNzgiDCD1RrEL0tv0ib9ATyqRA6iCm0iISemDLoV

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

  Using cached https://github.com/re-search/DocProduct/archive/v0.2.0_dev.zip
  Created wheel for docproduct: filename=docproduct-0.2.0-cp36-none-any.whl size=61649 sha256=23e76d96be60a98351ecda3f5189d4f25735590af20e4c99f5a73d08a1a24f2e
  Stored in directory: /tmp/pip-ephem-wheel-cache-j5ohxcwb/wheels/d6/ff/7e/7c265da92fd0d24132f65f8f7e28c63c8381f5478c32bd9135
Successfully built docproduct


In [0]:
#@title Downaload all model checkpoints, and question/answer data. Double click to see this code

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
import os
import requests

import urllib.request

# Download the file from `url` and save it locally under `file_name`:
urllib.request.urlretrieve('https://github.com/naver/biobert-pretrained/releases/download/v1.0-pubmed-pmc/biobert_v1.0_pubmed_pmc.tar.gz', 'BioBert.tar.gz')

if not os.path.exists('BioBertFolder'):
    os.makedirs('BioBertFolder')
    
import tarfile
tar = tarfile.open("BioBert.tar.gz")
tar.extractall(path='BioBertFolder/')
tar.close()

file_id = '1uCXv6mQkFfpw5txGnVCsl93Db7t5Z2mp'

download_file_from_google_drive(file_id, 'Float16EmbeddingsExpanded5-27-19.pkl')

file_id = 'https://onedrive.live.com/download?cid=9DEDF3C1E2D7E77F&resid=9DEDF3C1E2D7E77F%2132792&authkey=AEQ8GtkcDbe3K98'
    
urllib.request.urlretrieve( file_id, 'DataAndCheckpoint.zip')

if not os.path.exists('newFolder'):
    os.makedirs('newFolder')

import zipfile
zip_ref = zipfile.ZipFile('DataAndCheckpoint.zip', 'r')
zip_ref.extractall('newFolder')
zip_ref.close()

In [0]:
#@title Load model weights and Q&A data. Double click to see code

from docproduct.predictor import RetreiveQADoc

pretrained_path = 'BioBertFolder/biobert_v1.0_pubmed_pmc/'
# ffn_weight_file = None
bert_ffn_weight_file = 'newFolder/models/bertffn_crossentropy/bertffn'
embedding_file = 'Float16EmbeddingsExpanded5-27-19.pkl'

doc = RetreiveQADoc(pretrained_path=pretrained_path,
ffn_weight_file=None,
bert_ffn_weight_file=bert_ffn_weight_file,
embedding_file=embedding_file)

In [0]:
#@title Type in your question (512 word limit) and search search parameters

question_text = "I\u2019ve had chronic migraines for four years and they just keep getting worse.  It\u2019s to the point now that I have between 20 and 25 migraines a month. I\u2019ve had a CT scan which came back normal, I\u2019ve been on amitriptyline, topamax, and propranolol to try to prevent the migraines and I\u2019ve used maxalt and sumatriptan to break migraines at onset. I haven\u2019t had consistent relief with any medication, the 3 prophylactics only helped a few of my symptoms and the maxalt/ sumatriptan only succeeded in breaking my migraines about 25% of the time.  I just want to know if anyone can help me figure out what is wrong with me. I\u2019ll list my symptoms below.  -Sharp shooting pain shooting down jawline from mandibular joint and across cheekbones. Usually on only one side of my face but it changes sides randomly.  -Extreme pain in eyes, sensitivity to light and movement. See flashes of light even when eyes are closed.  -sharp pain starting at base of head/top of neck that runs up head and causes pain in eyes.  -Sharp pain in ears usually accompanied by ringing  Dizziness and nausea, occasionally vomiting  Muscle spasms, usually in hands or face  All pain worsens with any change in position; standing, sitting, rolling over while lying down.  Usually I\u2019ll know that I\u2019m going to get a migraine because it starts with the shooting face pain or the eye pain. The 3 prophylactics helped the facial pain but didn\u2019t reduce the number of migraines I had per month and they all gave me pretty bad side effects.  Side effects from medication  Topamax: felt drunk, nauseous, twitchy  Amitriptyline: extreme fatigue even into the next day when taken at night, confusion, balance issues, muscle spasms  Propranolol: nausea, hair loss, fatigue"  #@param {type:"string"}

search_similarity_by = 'answer'  #@param ['answer', "question"]

number_results_toReturn=10 #@param {type:"number"}

answer_only=True #@param ["False", "True"] {type:"raw"}

returned_results = doc.predict( question_text ,
                  search_by=search_similarity_by, topk=number_results_toReturn, answer_only=answer_only)

print('')
for jk in range(len(returned_results)):
    print("Result ", jk+1)
    print(returned_results[jk])
    print('')
